# Setup

In [1]:
import os

from dotenv import load_dotenv

from fixpoint.cache import ChatCompletionDiskTLRUCache
from fixpoint.utils.messages import umsg, smsg, umsg

In [2]:
from fixpoint_extras.services import formagent
from fixpoint_extras.services.formagent.setup import (
    setup_workflow
)
from fixpoint_extras.services.formagent.controllers.infogather import (
    InfoGatherer
)
from fixpoint_extras.workflows.imperative.form import Form

In [3]:
import logging
import sys
from fixpoint.logging import logger
logging.basicConfig(stream=sys.stdout)
logger.setLevel(logging.DEBUG)
logger.info("Info mode is on?")

INFO:fixpoint:Info mode is on?


In [4]:
assert load_dotenv('../.env')

In [5]:
cache = ChatCompletionDiskTLRUCache(
    './diskcache/',
    ttl_s=60*60*24*7 # one week
)

In [ ]:
wfctx = setup_workflow(
    os.environ['OPENAI_API_KEY'],
    model_name='gpt-3.5-turbo',
    cache=cache,
    # uncomment this is you want to proxy requests through our logging server to log them
    # openai_base_url='http://localhost:8081/v1/proxy/openai/',
    default_openai_headers={
        "Fixpoint-Inference-API-Key": os.environ['OPENAI_API_KEY'],
        "Fixpoint-API-Key": os.environ['FIXPOINT_API_KEY'],
    }
)

In [ ]:
invoice_questions_form = Form(
    id="invoice_questions",
    path="/invoice",
    form_schema=formagent.tasks.invoice.InvoiceQuestions
)

In [ ]:
info_gatherer = InfoGatherer(invoice_questions_form, wfctx.agents['main'])

# Run the steps

## Classify

In [ ]:
# wfctx.agent.set_cache_mode('normal')
# wfctx.agent.set_cache_mode('skip_lookup')
# wfctx.cache.clear()
formtype, completion = formagent.tasks.classify_form_type(
    wfctx,
    "I am making a form to charge a client for my consulting services",
    # cache_mode="skip_lookup"
)
print()
print(completion.chain_of_thought)
print(formtype.value)

## Invoice

In [ ]:
# wfctx.agents['main'].set_cache_mode("skip_lookup")
# wfctx.agent.set_cache_mode("normal")

# inv_qs, inv_compl = formagent.tasks.answer_invoice_questions(
#     wfctx,
#     "I am sending the invoice to Dylan at dylan@fixpoint.co"
# )

invoice_info, completion = formagent.tasks.invoice.gather_invoice_info(
    wfctx,
    info_gatherer,
    user_message="I am sending the invoice to Dylan at dylan@fixpoint.co"
)

In [ ]:
invoice_info

In [ ]:
info_gatherer._questions

In [ ]:
print(info_gatherer.format_questions())

In [ ]:
invoice_info, completion = formagent.tasks.invoice.gather_invoice_info(
    wfctx,
    info_gatherer,
    # TODO(dbmikus) we should be able to answer form invoice questions based on
    # the original classification message.
    user_message="The cost is $25k, and its for my AI/LLM consulting services."
)

In [ ]:
info_gatherer.is_complete()

In [ ]:
import pprint

pprint.pprint(info_gatherer.form.contents.model_dump())
print()
pprint.pprint([h.model_dump() for h in info_gatherer.info_history])

# Scratch

In [ ]:
import json

print(json.dumps(
    formagent.tasks.invoice.InvoiceQuestions.schema(),
    indent=2
))

In [ ]:
from collections import defaultdict

# defaultdict(defaultdict(dict))
x = defaultdict(lambda: defaultdict(dict))

In [ ]:
x['w1']['t1']['s1'] = 'hi'

In [ ]:
x['w1']['t1']['s1']

In [ ]:
from fixpoint import WorkflowRun

In [ ]:
w = WorkflowRun(workflow_id="hi", id="a-run")

In [ ]:
w

In [ ]:
w.id

In [ ]:
df = wfctx.memory.to_dataframe()
df